In [1]:
def combine_files_from_list(list_file, output_file):
    # Read file paths from the given file (one path per line)
    with open(list_file, "r", encoding="utf-8") as file:
        file_paths = [line.strip() for line in file if line.strip()]
    
    with open(output_file, "w", encoding="utf-8") as out_file:
        for path in file_paths:
            # Write the file path as a header
            out_file.write(f"File: {path}\n")
            out_file.write("=" * 60 + "\n")
            try:
                # Open and read the content of the current file
                with open(path, "r", encoding="utf-8") as f:
                    content = f.read()
                    out_file.write(content)
            except Exception as e:
                # Write an error message if the file can't be read
                out_file.write(f"Error reading file: {e}\n")
            out_file.write("\n" + "-" * 60 + "\n\n")

if __name__ == "__main__":
    # File that contains the list of paths (one per line) in the given format
    list_file = "file_paths.txt"
    # Output file where combined content will be saved
    output_file = "D:\Documents\E_Plus_2030_py\ZZZz\combined_files.txt"
    
    combine_files_from_list(list_file, output_file)
    print(f"Combined file created: {output_file}")


Combined file created: D:\Documents\E_Plus_2030_py\ZZZz\combined_files.txt


In [2]:
import sys

# Suppose check_files.py is in D:\Documents\E_Plus_2030_py\ZZZz
sys.path.append(r"D:\Documents\E_Plus_2030_py\ZZZz")

from check_files import check_number_of_columns_and_extract, check_number_of_rows_and_extract

# Columns check
check_number_of_columns_and_extract(
    file_list_path=r"D:\Documents\E_Plus_2030_py\ZZZz\file_list.txt",
    expected_columns=1,
    output_txt_path=r"D:\Documents\E_Plus_2030_py\ZZZz\columns_report.txt"
)

# Rows check
check_number_of_rows_and_extract(
    file_list_path=r"D:\Documents\E_Plus_2030_py\ZZZz\file_list.txt",
    expected_rows=1,
    output_txt_path=r"D:\Documents\E_Plus_2030_py\ZZZz\rows_report.txt"
)


In [3]:
import pandas as pd
import random

def get_building_data_with_random_columns(file_path, building_id_to_filter, num_random_columns):
    """
    Reads a CSV file, filters by BuildingID, and returns the first two columns
    along with a specified number of random columns.

    Args:
        file_path (str): The path to the CSV file.
        building_id_to_filter (int or str): The BuildingID to filter by.
        num_random_columns (int): The number of random columns to select.

    Returns:
        pandas.DataFrame: A DataFrame containing the selected data,
                          or None if an error occurs.
    """
    try:
        # Attempt to read with comma separator first, as per previous findings
        print(f"Attempting to read '{file_path}' with separator: ','")
        df = pd.read_csv(file_path, sep=',')

        # --- Diagnostic: Print detected column names ---
        print("\nDetected column names by pandas (after attempting comma separation):")
        original_columns = df.columns.tolist()
        print(original_columns)
        # --- End Diagnostic ---

        # Check if parsing was successful (more than one column and 'BuildingID' likely present)
        if not (len(original_columns) > 1 and any('BuildingID' in col for col in original_columns)):
            print("\nComma separation might not have worked. Trying tab separation...")
            df = pd.read_csv(file_path, sep='\t')
            print(f"Attempting to read '{file_path}' with separator: '\\t'")
            print("\nDetected column names by pandas (after attempting tab separation):")
            original_columns = df.columns.tolist()
            print(original_columns)


        # Clean column names: strip leading/trailing whitespace
        df.columns = df.columns.str.strip()
        print("\nColumn names after stripping whitespace:")
        cleaned_columns = df.columns.tolist()
        print(cleaned_columns)
        # --- End Diagnostic ---

        building_id_column_name = 'BuildingID'

        if building_id_column_name not in df.columns:
            print(f"\nError: Column '{building_id_column_name}' not found in the CSV after parsing with available delimiters.")
            print(f"Actual columns found: {df.columns.tolist()}")
            print(f"Please ensure that '{building_id_column_name}' matches one of the detected column names exactly (including case), "
                  f"and that the file's delimiter (e.g., ',', '\\t', ';') is correctly identified by the script or specified.")
            return None

        df[building_id_column_name] = df[building_id_column_name].astype(str)
        building_df = df[df[building_id_column_name] == str(building_id_to_filter)]

        if building_df.empty:
            print(f"\nNo data found for {building_id_column_name}: {building_id_to_filter}")
            return None

        if len(df.columns) < 2:
            print("\nError: The CSV file has fewer than 2 columns after parsing.")
            return None
        first_two_columns = df.columns[:2].tolist()
        remaining_columns = df.columns[2:].tolist()

        if num_random_columns > len(remaining_columns):
            print(f"\nWarning: Requested {num_random_columns} random columns, "
                  f"but only {len(remaining_columns)} are available after the first two.")
            num_random_columns = len(remaining_columns)

        randomly_selected_columns = []
        if num_random_columns > 0 and remaining_columns:
            randomly_selected_columns = random.sample(remaining_columns, num_random_columns)

        final_columns_to_select = []
        for col in first_two_columns + randomly_selected_columns:
            if col in building_df.columns:
                final_columns_to_select.append(col)
            else:
                print(f"\nWarning: Column '{col}' was expected but not found in the filtered data. Skipping.")
        
        if not final_columns_to_select:
            print("\nError: No columns were selected. Check column names and data.")
            return None
            
        result_df = building_df[final_columns_to_select]
        return result_df

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{file_path}' is empty or not a valid CSV.")
        return None
    except Exception as e:
        print(f"\nAn unexpected error occurred during CSV reading or initial processing: {e}")
        return None

if __name__ == "__main__":
    # --- Configuration ---
    # IMPORTANT: Replace this with the actual path to your CSV file.
    csv_file_path = r"D:\Documents\E_Plus_2030_py\output\cf868574-d471-47e3-99a4-231774bd7f36\results\merged_as_is.csv"
    # csv_file_path = "merged_as_is.csv" # Or just the filename if it's in the same directory

    target_building_id_val = 0  # The BuildingID you want to filter by (e.g., 0)
    number_of_random_cols = 5  # How many random columns you want
    
    # Specify the output text file name
    output_txt_file_path = f"building_{target_building_id_val}_random_cols_output.txt"
    # --- End Configuration ---

    selected_data = get_building_data_with_random_columns(csv_file_path, target_building_id_val, number_of_random_cols)

    if selected_data is not None and not selected_data.empty:
        print(f"\nData for BuildingID {target_building_id_val} with {number_of_random_cols} random columns:")
        # Print to console
        print(selected_data.to_string())

        # Save to text file
        try:
            with open(output_txt_file_path, 'w') as f:
                f.write(selected_data.to_string())
            print(f"\nSuccessfully saved data to '{output_txt_file_path}'")
        except Exception as e:
            print(f"\nError saving data to text file: {e}")
            
    elif selected_data is not None and selected_data.empty:
        print(f"\nNo data matched the criteria for BuildingID {target_building_id_val} (or the DataFrame was empty after processing).")
    else:
        print("\nNo data was processed or returned from the function.")



Attempting to read 'D:\Documents\E_Plus_2030_py\output\cf868574-d471-47e3-99a4-231774bd7f36\results\merged_as_is.csv' with separator: ','

Detected column names by pandas (after attempting comma separation):
['BuildingID', 'VariableName', '01/01 00:00:00', '01/01  01:00:00', '01/01  02:00:00', '01/01  03:00:00', '01/01  04:00:00', '01/01  05:00:00', '01/01  06:00:00', '01/01  07:00:00', '01/01  08:00:00', '01/01  09:00:00', '01/01  10:00:00', '01/01  11:00:00', '01/01  12:00:00', '01/01  13:00:00', '01/01  14:00:00', '01/01  15:00:00', '01/01  16:00:00', '01/01  17:00:00', '01/01  18:00:00', '01/01  19:00:00', '01/01  20:00:00', '01/01  21:00:00', '01/01  22:00:00', '01/01  23:00:00', '01/02 00:00:00', '01/02  01:00:00', '01/02  02:00:00', '01/02  03:00:00', '01/02  04:00:00', '01/02  05:00:00', '01/02  06:00:00', '01/02  07:00:00', '01/02  08:00:00', '01/02  09:00:00', '01/02  10:00:00', '01/02  11:00:00', '01/02  12:00:00', '01/02  13:00:00', '01/02  14:00:00', '01/02  15:00:00', '01

In [15]:
import pandas as pd
import random

def get_optimized_building_data(file_path, building_id_to_filter, num_random_columns):
    """
    Optimized function to read specific columns from a CSV file, filter by BuildingID,
    and return the first two columns along with a specified number of random data columns.

    Args:
        file_path (str): The path to the CSV file.
        building_id_to_filter (str or int): The BuildingID to filter by.
        num_random_columns (int): The number of random data columns to select.

    Returns:
        pandas.DataFrame: A DataFrame containing the selected data,
                          or None if an error occurs.
    """
    print(f"Starting optimized data retrieval for BuildingID: {building_id_to_filter} from '{file_path}'")
    
    detected_separator = None
    headers = []

    # --- Step 1: Detect Separator and Get Headers ---
    try:
        # Try with comma first
        print("Attempting to detect separator and read headers (trying comma)...")
        headers_df_comma = pd.read_csv(file_path, sep=',', nrows=0) # Read only header
        if len(headers_df_comma.columns) > 1:
            detected_separator = ','
            headers = [col.strip() for col in headers_df_comma.columns.tolist()]
            print(f"Detected comma (',') as separator. Headers: {headers}")
        else:
            # If comma didn't work well (e.g., one long column name), try tab
            print("Comma separation resulted in single column, trying tab...")
            headers_df_tab = pd.read_csv(file_path, sep='\t', nrows=0) # Read only header
            if len(headers_df_tab.columns) > 1:
                detected_separator = '\t'
                headers = [col.strip() for col in headers_df_tab.columns.tolist()]
                print(f"Detected tab ('\\t') as separator. Headers: {headers}")
            else: # Fallback if tab also results in single column
                 print("Tab separation also resulted in single column. Assuming comma for the single long header.")
                 headers = [col.strip() for col in headers_df_comma.columns[0].split(',')]
                 if len(headers) > 1 :
                    detected_separator = ',' # It was comma, but pandas initially failed to split.
                    print(f"Manually split headers assuming comma. Headers: {headers}")
                 else:
                    print("Error: Could not determine a valid separator or parse headers correctly.")
                    print(f"Headers read with comma (potentially unsplit): {headers_df_comma.columns.tolist()}")
                    print(f"Headers read with tab (potentially unsplit): {headers_df_tab.columns.tolist()}")
                    return None
        
        if not detected_separator:
            print("Error: Failed to auto-detect a suitable CSV separator (tried comma and tab).")
            print("Please ensure your CSV is comma or tab-separated, or specify the separator manually if needed.")
            return None

    except Exception as e:
        print(f"Error reading headers: {e}")
        return None

    if not headers:
        print("Error: No headers found in the file.")
        return None
    
    print(f"\nCleaned headers: {headers}")

    # --- Step 2: Identify Columns to Keep ---
    if len(headers) < 1:
        print("Error: CSV file has no columns.")
        return None
    
    # Assuming the first column is always BuildingID and the second is VariableName
    # This aligns with "first two columns" requirement
    col_building_id_name = headers[0]
    
    columns_to_always_keep = []
    if len(headers) >= 1:
        columns_to_always_keep.append(headers[0]) # First column (BuildingID)
    if len(headers) >= 2:
        columns_to_always_keep.append(headers[1]) # Second column (VariableName)
    else: # Only one column in file
        print("Warning: CSV file has only one column. Will use it as BuildingID.")

    print(f"Columns to always keep: {columns_to_always_keep}")

    data_column_candidates = headers[len(columns_to_always_keep):]
    print(f"Candidate data columns for random selection: {data_column_candidates}")

    selected_data_columns = []
    if num_random_columns > 0 and data_column_candidates:
        if num_random_columns > len(data_column_candidates):
            print(f"Warning: Requested {num_random_columns} random columns, "
                  f"but only {len(data_column_candidates)} data columns are available. Selecting all available.")
            num_random_columns = len(data_column_candidates)
        selected_data_columns = random.sample(data_column_candidates, num_random_columns)
        print(f"Randomly selected data columns: {selected_data_columns}")

    # Final list of columns to load from the CSV
    cols_to_load_final = list(dict.fromkeys(columns_to_always_keep + selected_data_columns)) # dict.fromkeys to remove duplicates and preserve order
    print(f"Final columns to load from CSV: {cols_to_load_final}")

    if not cols_to_load_final:
        print("Error: No columns selected for loading. Check headers and selection logic.")
        return None
    if col_building_id_name not in cols_to_load_final:
        print(f"Error: Identified BuildingID column '{col_building_id_name}' is not in the final list of columns to load.")
        return None


    # --- Step 3: Load Only Specified Columns and Filter Rows ---
    try:
        print(f"\nReading specified columns from '{file_path}' using separator '{detected_separator}'...")
        # Specify dtype for BuildingID column to ensure it's read as string for consistent filtering
        dtype_spec = {col_building_id_name: str} 
        
        df_selected_cols = pd.read_csv(
            file_path,
            sep=detected_separator,
            usecols=cols_to_load_final,
            dtype=dtype_spec 
        )
        print(f"Successfully loaded {len(df_selected_cols)} rows with {len(df_selected_cols.columns)} selected columns.")

        # Filter by BuildingID
        # Ensure the building_id_to_filter is also a string for comparison
        result_df = df_selected_cols[df_selected_cols[col_building_id_name] == str(building_id_to_filter)]
        print(f"Filtered down to {len(result_df)} rows for BuildingID '{building_id_to_filter}'.")
        
        if result_df.empty:
            print(f"No data found matching BuildingID: {building_id_to_filter} after loading selected columns.")
            return result_df # Return empty DataFrame to indicate no match

        return result_df

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found during the main read.")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{file_path}' is empty or not a valid CSV.")
        return None
    except ValueError as ve:
        print(f"ValueError during data loading or processing: {ve}")
        print("This might happen if `usecols` contains a column not in the CSV or due to dtype issues.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during main data loading or filtering: {e}")
        return None

if __name__ == "__main__":
    # --- Configuration ---
    # IMPORTANT: Replace this with the actual path to your CSV file.
    csv_file_path = r"D:\Documents\E_Plus_2030_py\output\399a658c-b28a-4871-966a-10cf1bfedce0\results\merged_as_is.csv"
    # Example for testing:
    # csv_file_path = "merged_as_is.csv" 

    target_building_id_val = "1"  # The BuildingID you want to filter by (e.g., "0" or 0)
    number_of_random_cols = 15    # How many random data columns you want (in addition to the first two)
    
    # Specify the output text file name
    output_txt_file_path = f"building_{target_building_id_val}_random_cols_output.txt"
    # --- End Configuration ---

    print("--- Script Start ---")
    selected_data = get_optimized_building_data(csv_file_path, target_building_id_val, number_of_random_cols)

    if selected_data is not None:
        if not selected_data.empty:
            print(f"\nData for BuildingID {target_building_id_val} with {number_of_random_cols} random columns (and first two):")
            # Print to console
            print(selected_data.to_string())

            # Save to text file
            try:
                with open(output_txt_file_path, 'w', encoding='utf-8') as f: # Added encoding
                    f.write(f"Data for BuildingID {target_building_id_val}:\n")
                    f.write(selected_data.to_string())
                print(f"\nSuccessfully saved data to '{output_txt_file_path}'")
            except Exception as e:
                print(f"\nError saving data to text file: {e}")
        else:
            print(f"\nNo data matched the criteria for BuildingID {target_building_id_val} after processing.")
    else:
        print("\nNo data was processed due to an error or no data returned from the function.")
    print("--- Script End ---")



--- Script Start ---
Starting optimized data retrieval for BuildingID: 1 from 'D:\Documents\E_Plus_2030_py\output\399a658c-b28a-4871-966a-10cf1bfedce0\results\merged_as_is.csv'
Attempting to detect separator and read headers (trying comma)...
Detected comma (',') as separator. Headers: ['BuildingID', 'VariableName', '01/01 00:00:00', '01/01  01:00:00', '01/01  02:00:00', '01/01  03:00:00', '01/01  04:00:00', '01/01  05:00:00', '01/01  06:00:00', '01/01  07:00:00', '01/01  08:00:00', '01/01  09:00:00', '01/01  10:00:00', '01/01  11:00:00', '01/01  12:00:00', '01/01  13:00:00', '01/01  14:00:00', '01/01  15:00:00', '01/01  16:00:00', '01/01  17:00:00', '01/01  18:00:00', '01/01  19:00:00', '01/01  20:00:00', '01/01  21:00:00', '01/01  22:00:00', '01/01  23:00:00', '01/02 00:00:00', '01/02  01:00:00', '01/02  02:00:00', '01/02  03:00:00', '01/02  04:00:00', '01/02  05:00:00', '01/02  06:00:00', '01/02  07:00:00', '01/02  08:00:00', '01/02  09:00:00', '01/02  10:00:00', '01/02  11:00:00', 

In [2]:
import pandas as pd

def get_unique_building_types(file_paths, column_name="building_type"):
    """
    Extracts unique values from a specified column in multiple Excel files.

    Args:
        file_paths (list): A list of strings, where each string is the path to an Excel file.
        column_name (str): The header name of the column to extract unique values from.

    Returns:
        set: A set containing all unique values from the specified column
             across all provided files. Returns an empty set if an error occurs
             or the column is not found.
    """
    unique_values = set()

    for file_path in file_paths:
        try:
            # Read the Excel file
            # The engine='openpyxl' is specified for .xlsx files
            df = pd.read_excel(file_path, engine='openpyxl')

            # Check if the column exists in the DataFrame
            if column_name in df.columns:
                # Add unique values from the column to the set
                # .dropna() is used to remove any NaN (Not a Number) values
                # which can occur if some cells in the column are empty.
                unique_values.update(df[column_name].dropna().unique())
            else:
                print(f"Warning: Column '{column_name}' not found in file: {file_path}")

        except FileNotFoundError:
            print(f"Error: File not found at path: {file_path}")
        except Exception as e:
            print(f"An error occurred while processing file {file_path}: {e}")

    return unique_values

# --- Main execution ---
if __name__ == "__main__":
    # Define the paths to your Excel files
    # IMPORTANT: Replace these with the actual paths to your files if they are different.
    # Using raw strings (r"...") for Windows paths to handle backslashes correctly.
    file1_path = r"D:\Documents\E_Plus_2030_py\lookup_pys\envelop_nonres6.xlsx"
    file2_path = r"D:\Documents\E_Plus_2030_py\lookup_pys\envelop_res6.xlsx"

    files_to_process = [file1_path, file2_path]
    target_column = "building_type"

    print(f"Attempting to extract unique values from column '{target_column}' in files:")
    for f_path in files_to_process:
        print(f"- {f_path}")
    print("-" * 30)

    unique_building_types = get_unique_building_types(files_to_process, target_column)

    if unique_building_types:
        print("\nUnique building types found:")
        for b_type in sorted(list(unique_building_types)): # Sorted for consistent output
            print(f"- {b_type}")
    else:
        print("\nNo unique building types found or an error occurred.")


Attempting to extract unique values from column 'building_type' in files:
- D:\Documents\E_Plus_2030_py\lookup_pys\envelop_nonres6.xlsx
- D:\Documents\E_Plus_2030_py\lookup_pys\envelop_res6.xlsx
------------------------------

Unique building types found:
- Accommodation Function
- Apartment
- Cell Function
- Corner House
- Detached House
- Education Function
- Healthcare Function
- Industrial Function
- Meeting Function
- Office Function
- Other Use Function
- Retail Function
- Sport Function
- Terrace or Semi-detached House
- Two-and-a-half-story House


In [ ]:
from eppy.modeleditor import IDF

# Set your EnergyPlus paths here
idd_file = r'D:\Documents\E_Plus_2030_py\EnergyPlus\Energy+.idd'
idf_file = r'D:\Documents\E_Plus_2030_py\EnergyPlus\Min.idf'

# Initialize IDF
IDF.setiddname(idd_file)
idf = IDF(idf_file)




# Shading parameters for testing
shading_params = {
    "blind_name": "TestExternalLouvers",
    "slat_orientation": "Horizontal",
    "slat_width": 0.025,
    "slat_separation": 0.020,
    "slat_thickness": 0.001,
    "slat_angle_deg": 45.0,
    "slat_conductivity": 160.0,
    "slat_beam_solar_transmittance": 0.0,
    "slat_beam_solar_reflectance": 0.7,
    "slat_diffuse_solar_transmittance": 0.0,
    "slat_diffuse_solar_reflectance": 0.7,
    "slat_beam_visible_transmittance": 0.0,
    "slat_beam_visible_reflectance": 0.7,
    "slat_diffuse_visible_transmittance": 0.0,
    "slat_diffuse_visible_reflectance": 0.7,
    "slat_ir_transmittance": 0.0,
    "slat_ir_emissivity": 0.9,
    "blind_to_glass_distance": 0.05,  
    "blind_top_opening_multiplier": 0.5,
    "blind_bottom_opening_multiplier": 0.0,
    "blind_left_side_opening_multiplier": 0.5,
    "blind_right_side_opening_multiplier": 0.5,
    # Corrected field name:
    "type_of_slat_angle_control_for_blinds": "FixedSlatAngle",
    "shading_control_type": "AlwaysOn",
}

window_name = "Zone2_Wall_3_window"
zone_name = "Zone2"  # Adjust according to your model if needed

# Create WindowMaterial:Blind
try:
    blind_mat = idf.newidfobject("WINDOWMATERIAL:BLIND")
    blind_mat.Name = shading_params["blind_name"]
    blind_mat.Slat_Orientation = shading_params["slat_orientation"]
    blind_mat.Slat_Width = shading_params["slat_width"]
    blind_mat.Slat_Separation = shading_params["slat_separation"]
    blind_mat.Slat_Thickness = shading_params["slat_thickness"]
    blind_mat.Slat_Angle = shading_params["slat_angle_deg"]
    blind_mat.Slat_Conductivity = shading_params["slat_conductivity"]

    blind_mat.Slat_Beam_Solar_Transmittance = shading_params["slat_beam_solar_transmittance"]
    blind_mat.Front_Side_Slat_Beam_Solar_Reflectance = shading_params["slat_beam_solar_reflectance"]
    blind_mat.Back_Side_Slat_Beam_Solar_Reflectance = shading_params["slat_beam_solar_reflectance"]

    blind_mat.Slat_Diffuse_Solar_Transmittance = shading_params["slat_diffuse_solar_transmittance"]
    blind_mat.Front_Side_Slat_Diffuse_Solar_Reflectance = shading_params["slat_diffuse_solar_reflectance"]
    blind_mat.Back_Side_Slat_Diffuse_Solar_Reflectance = shading_params["slat_diffuse_solar_reflectance"]

    blind_mat.Slat_Beam_Visible_Transmittance = shading_params["slat_beam_visible_transmittance"]
    blind_mat.Front_Side_Slat_Beam_Visible_Reflectance = shading_params["slat_beam_visible_reflectance"]
    blind_mat.Back_Side_Slat_Beam_Visible_Reflectance = shading_params["slat_beam_visible_reflectance"]

    blind_mat.Slat_Diffuse_Visible_Transmittance = shading_params["slat_diffuse_visible_transmittance"]
    blind_mat.Front_Side_Slat_Diffuse_Visible_Reflectance = shading_params["slat_diffuse_visible_reflectance"]
    blind_mat.Back_Side_Slat_Diffuse_Visible_Reflectance = shading_params["slat_diffuse_visible_reflectance"]

    blind_mat.Slat_Infrared_Hemispherical_Transmittance = shading_params["slat_ir_transmittance"]
    blind_mat.Front_Side_Slat_Infrared_Hemispherical_Emissivity = shading_params["slat_ir_emissivity"]
    blind_mat.Back_Side_Slat_Infrared_Hemispherical_Emissivity = shading_params["slat_ir_emissivity"]

    blind_mat.Blind_to_Glass_Distance = shading_params["blind_to_glass_distance"]

    blind_mat.Blind_Top_Opening_Multiplier = shading_params["blind_top_opening_multiplier"]
    blind_mat.Blind_Bottom_Opening_Multiplier = shading_params["blind_bottom_opening_multiplier"]
    blind_mat.Blind_Left_Side_Opening_Multiplier = shading_params["blind_left_side_opening_multiplier"]
    blind_mat.Blind_Right_Side_Opening_Multiplier = shading_params["blind_right_side_opening_multiplier"]

    print("✅ Blind material object created successfully:")
    print(blind_mat)

except Exception as e:
    print(f"❌ Error creating WindowMaterial:Blind object: {e}")

# Create WindowShadingControl (fixed the field name here!)
try:
    shading_ctrl = idf.newidfobject("WINDOWSHADINGCONTROL")
    shading_ctrl.Name = f"ShadingCtrl_{window_name}"
    shading_ctrl.Zone_Name = zone_name
    shading_ctrl.Shading_Type = "ExteriorBlind"
    shading_ctrl.Shading_Control_Type = shading_params["shading_control_type"]
    shading_ctrl.Shading_Device_Material_Name = shading_params["blind_name"]
    
    # Correct field name:
    shading_ctrl.Type_of_Slat_Angle_Control_for_Blinds = shading_params["type_of_slat_angle_control_for_blinds"]
    #shading_ctrl.Fixed_Slat_Angle = shading_params["slat_angle_deg"]
    
    # Assign fenestration surface
    shading_ctrl.Fenestration_Surface_1_Name = window_name

    print("\n✅ WindowShadingControl object created successfully:")
    print(shading_ctrl)

except Exception as e:
    print(f"❌ Error creating WindowShadingControl object: {e}")




✅ Blind material object created successfully:

WINDOWMATERIAL:BLIND,
    TestExternalLouvers,      !- Name
    Horizontal,               !- Slat Orientation
    0.025,                    !- Slat Width
    0.02,                     !- Slat Separation
    0.001,                    !- Slat Thickness
    45,                       !- Slat Angle
    160,                      !- Slat Conductivity
    0,                        !- Slat Beam Solar Transmittance
    0.7,                      !- Front Side Slat Beam Solar Reflectance
    0.7,                      !- Back Side Slat Beam Solar Reflectance
    0,                        !- Slat Diffuse Solar Transmittance
    0.7,                      !- Front Side Slat Diffuse Solar Reflectance
    0.7,                      !- Back Side Slat Diffuse Solar Reflectance
    0,                        !- Slat Beam Visible Transmittance
    0.7,                      !- Front Side Slat Beam Visible Reflectance
    0.7,                      !- Back Side Slat

: 